In [7]:
from fred import Fred
import numpy as np
import pandas as pd
from datetime import datetime
import time

# FRED API key
# key = 'enter your key'
%run -i fred_api_key

## Getting Data from FRED ##
fr = Fred(api_key = key, response_type = 'df')

# Setting data parameters
start = datetime(1990, 1, 1)
end = datetime(2019, 4, 1)

params = {'observation_start': start.strftime('%Y-%m-%d'),
          'observation_end': end.strftime('%Y-%m-%d')}

# Load list of data series
fseries = pd.read_csv("FRED_data_series.csv")

In [8]:
def quarterMean(series, dates):
    ret = []
    for start, end in zip(dates[:-1], dates[1:]):
        batch = series.loc[(series.date >= start) & (series.date < end), 'value']
        shrink = np.mean(batch)
        ret.append(shrink)
    return ret

def growthRate(series):
    return np.exp(np.diff(np.log(series))) - 1

class DataGetter:
    include = True
    def __init__(self, code, dates, method, params):
        self.code = code
        self.dates = dates
        self.method = method
        self.params = params
        self.start = fr.series.details(code).observation_start[0]

    def levelShrink(self, dates, series):
        ret = []
        for start, end in zip(dates[:-1], dates[1:]):
            batch = series.loc[(series.date >= start) & (series.date < end), 'value']
            if (batch.isnull().all()):
                self.include = False
                break
            if (all((x == 0) | np.isnan(x) for x in batch)):
                self.include = False
                break
            shrink = np.prod(1 + batch) - 1
            ret.append(shrink)
        return ret

    def percentShrink(self, dates, series):
        ret = []
        for start, end in zip(dates[:-1], dates[1:]):
            batch = series.loc[(series.date >= start) & (series.date < end), 'value']
            if (batch.isnull().all()):
                self.include = False
                break
            if (all((x == 0) | np.isnan(x) for x in batch)):
                self.include = False
                break
            shrink = np.sum(np.diff(batch))
            ret.append(shrink)
        return ret

    def getter(self):
        series = fr.series.observations(self.code, params = self.params)
        series.dropna(axis = 0, inplace = True)
        if(method):
            series = self.percentShrink(self.dates, series)
        else:
            series = pd.DataFrame({'date' : series.date[1:], 'value': growthRate(series.value)})
            series = self.levelShrink(self.dates, series)
        return series

In [ ]:
# Loading GDP
gdp = fr.series.observations("GDPC1", params = params)
gdp.drop(labels = ['realtime_end', 'realtime_start'], axis = 1, inplace = True)
gdp.rename(columns = {'value' : 'gdp'}, inplace = True)
gdp.gdp

# Retrieving data from Fred
n = fseries.shape[0]
dates = gdp.date

data = pd.DataFrame({'date' : dates[1:]})

for i in range(700,n):
    code = fseries.loc[i, 'file']
    method = fseries.loc[i, 'method']
    
    dg = DataGetter(code, dates, method, params)
    series = dg.getter()
    
    if ((dg.start <= start) & dg.include):
        data[code] = series
    if (i % 100  == 0): 
        print(i)
    time.sleep(.5)

# adding in zero-indexed data
data['MEIM683SFRBCHI'] = quarterMean(fr.series.observations('MEIM683SFRBCHI', params = params), dates)
data['RMEIM683SFRBCHI'] = quarterMean(fr.series.observations('RMEIM683SFRBCHI', params = params), dates)

# adding lagged GDP
lag1 = {'observation_start': "1989-10-01",
        'observation_end': "2018-07-01"}

lag2 = {'observation_start': "1989-07-01",
        'observation_end': "2018-04-01"}

data['GDPLAG1'] = growthRate(fr.series.observations('GDP', params = lag1).value)
data['GDPLAG2'] = growthRate(fr.series.observations('GDP', params = lag2).value)